# Inference Time Study

In *base.py* file (mmengine) with class BaseModel add **imports** 

```python
        import time
        from demo.json_handler import JSONHandler
```

and **the following** (in test_step):

```python
        begin = time.time()
        data = self.data_preprocessor(data, False)
        end = time.time()

        out_file = '/home/michele*/iac_code/michele_mmdet3d/data/minerva_polimove/inference_times.json'
        handler = JSONHandler(out_file, True)
        handler.update_dictionary({'Pre-processing delta_t': (end-begin)})

        return self._run_forward(data, mode='predict')  # type: ignore
```

In [112]:
#################################################################################################################
#                                      INFERENCE AND SAVE THE DATA                                              #
#################################################################################################################

# Needed imports
import sys
from json_handler import JSONHandler
import time


# Add the new paths separately
new_path1 = "/home/michele/.local/lib/python3.10/site-packages/"
new_path2 = "/home/michele/iac_code/michele_mmdet3d/"
if not new_path1 in sys.path:
    sys.path.insert(1, new_path1)
if not new_path2 in sys.path:
    sys.path.insert(1, new_path2)
# Import the fucking wicked class
from mmdet3d.apis import LidarDet3DInferencer


# Initialize the actual inferencer class
# inferencer = LidarDet3DInferencer(model='/home/michele/iac_code/michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py', 
#                                   weights='/home/michele/iac_code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth')
inferencer = LidarDet3DInferencer(model='/home/michele/iac_code/michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py', 
                                  weights='/home/michele/iac_code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth',
                                  show_progress=False)


# Read the files in validation list
val_list_txt_file = "/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/ImageSets/val.txt"
with open(val_list_txt_file, 'r') as file:
    val_file_names = [line.strip() for line in file]
print(f"Total validation point_clouds: {len(val_file_names)}")


# Create the list of inputs, suitable for the inferencer
inputs = []
for i, name in enumerate(val_file_names):
    inputs.append(dict(points=("/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/training/velodyne/"+name+".bin")))


# Create an instance JSONHandler
out_file = '/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/inference_times.json'
handler = JSONHandler(out_file)
# Remove the file if already existing
handler.reset()
# Do the actual inference
results = []
for i, input in enumerate(inputs):
    begin = time.time()
    handler.add_dictionary({'Everything start': begin})
    results.append(inferencer(input))
    end = time.time()
    handler.update_dictionary({'Everything end': end})

Loads checkpoint by local backend from path: /home/michele/iac_code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth
Total validation point_clouds: 35


In [113]:
for element in handler.read_json_file():
    print(element)

dictionary_list = handler.read_json_file()
print(f"\nThere are {len(dictionary_list)} elements in total")

{'Everything start': 1728851537.4538245, 'Voxel encoder start': 1728851537.4808736, 'Voxel encoder delta_t': 0.0005962848663330078, 'Middle encoder delta_t': 0.0009686946868896484, 'Backbone delta_t': 0.0019385814666748047, 'Neck delta_t': 0.0003006458282470703, 'Post-processing delta_t': 0.034395694732666016, 'Everything end': 1728851537.5246177}
{'Everything start': 1728851537.5247326, 'Voxel encoder start': 1728851537.558655, 'Voxel encoder delta_t': 0.0005500316619873047, 'Middle encoder delta_t': 0.0012118816375732422, 'Backbone delta_t': 0.0007183551788330078, 'Neck delta_t': 0.0002148151397705078, 'Post-processing delta_t': 0.034605979919433594, 'Everything end': 1728851537.6024406}
{'Everything start': 1728851537.6025949, 'Voxel encoder start': 1728851537.6397586, 'Voxel encoder delta_t': 0.0005269050598144531, 'Middle encoder delta_t': 0.001026153564453125, 'Backbone delta_t': 0.0007579326629638672, 'Neck delta_t': 0.0002243518829345703, 'Post-processing delta_t': 0.0346887111

In [114]:
#################################################################################################################
#                                  STUDY OF BASE_MODEL (NOT ACCESSIBLE)                                         #
#################################################################################################################

start = handler.get_parameter('Everything start')
end = handler.get_parameter('Voxel encoder start')
estimated_delta = []
for i in range(len(start)):
    estimated_delta.append( end[i]-start[i] )

compute_error = False

if compute_error:
    actual_delta = handler.get_parameter('Pre-processing delta_t')
    error = []
    error_abs = []
    for i in range(len(actual_delta)):
        error.append( actual_delta[i]-estimated_delta[i] )
        error_abs.append( abs( actual_delta[i]-estimated_delta[i] ) )

av_est_deltaT = sum(estimated_delta)/len(estimated_delta)
print(f"Average value of the estimated delta_t: {av_est_deltaT}")
if compute_error:
    print(f"Average value of the error: {sum(error)/len(error)}")
    av_abs_error = sum(error_abs)/len(error_abs)
    print(f"Average value for the absolute error: {av_abs_error}")
    av_act_deltat = sum(actual_delta)/len(actual_delta)
    print(f"The percentage of error is {av_abs_error/av_est_deltaT*100}. The average actual delta_t is {av_act_deltat}")

Average value of the estimated delta_t: 0.03296135493687221


In [115]:
#################################################################################################################
#                                            GENERAL STUDY OF TIME                                              #
#################################################################################################################

start_everything = handler.get_parameter('Everything start')
start_voxel = handler.get_parameter('Voxel encoder start')
delta_voxel = handler.get_parameter('Voxel encoder delta_t')
delta_middle = handler.get_parameter('Middle encoder delta_t')
delta_backbone = handler.get_parameter('Backbone delta_t')
delta_neck = handler.get_parameter('Neck delta_t')
delta_post = handler.get_parameter('Post-processing delta_t')
end_everything = handler.get_parameter('Everything end')
handler.reset()

# Total time
delta_total = []
for i in range(len(start_everything)):
    delta_total.append(end_everything[i]-start_everything[i])
estimated_total = sum(delta_total)/len(delta_total)
print(f"Average total time: {estimated_total*1e3:.2f}ms which would be {1/estimated_total:.2f}Hz!")
# Preprocessing
delta_pre = []
for i in range(len(start_everything)):
    delta_pre.append(start_voxel[i]-start_everything[i])
estimated_pre = sum(delta_pre)/len(delta_pre)
print(f"\tAverage pre-processing time:\t{estimated_pre*1e3:.2f}ms which means\t{100*estimated_pre/estimated_total:.2f} over the total. \
      \n\t\tUsually 20% higher than actual value, which would be {estimated_pre*0.8*1e3:.2f}ms")
# Voxel encoder
estimated_voxel = sum(delta_voxel)/len(delta_voxel)
print(f"\tAverage voxel encoder time:\t{estimated_voxel*1e3:.2f}ms which means\t{100*estimated_voxel/estimated_total:.2f} over the total.")
# Middle encoder
estimated_middle = sum(delta_middle)/len(delta_middle)
print(f"\tAverage middle encoder time:\t{estimated_middle*1e3:.2f}ms which means\t{100*estimated_middle/estimated_total:.2f} over the total.")
# Backbone
estimated_backbone = sum(delta_backbone)/len(delta_backbone)
print(f"\tAverage backbone time:\t\t{estimated_backbone*1e3:.2f}ms which means\t{100*estimated_backbone/estimated_total:.2f} over the total.")
# Neck
estimated_neck = sum(delta_neck)/len(delta_neck)
print(f"\tAverage neck time:\t\t{estimated_neck*1e3:.2f}ms which means\t{100*estimated_neck/estimated_total:.2f} over the total.")
# Post-processing
estimated_post = sum(delta_post)/len(delta_post)
print(f"\tAverage post-processing time:\t{estimated_post*1e3:.2f}ms which means\t{100*estimated_post/estimated_total:.2f} over the total.")
# Final comment
sum_all_0 = estimated_pre + estimated_voxel + estimated_middle + estimated_backbone + estimated_neck + estimated_post
sum_all_1 = estimated_pre*0.8 + estimated_voxel + estimated_middle + estimated_backbone + estimated_neck + estimated_post
sum_inference = estimated_voxel + estimated_middle + estimated_backbone + estimated_neck
print(f"\nBy summing all the \"sub-components\" we get:\
      \n\tA total time of {sum_all_0*1e3:.2f}ms ({1/sum_all_0:.2f}Hz) which means\t{100*sum_all_0/estimated_total:.2f}% over the total of\t{estimated_total*1e3:.2f}ms\
      \nand by considering the \"real\" value for the preprocessing (20% less), we get:\
      \n\tA total time of {sum_all_1*1e3:.2f}ms ({1/sum_all_1:.2f}Hz) which means\t{100*sum_all_1/estimated_total:.2f}% over the total of\t{estimated_total*1e3:.2f}ms\
      \nwhile the inference (from voxel encoder to neck) accounts for:\
      \n\tA total time of {sum_inference*1e3:.2f}ms which means\t\t{100*sum_inference/estimated_total:.2f}% over the total of\t\t{estimated_total*1e3:.2f}ms")

Average total time: 76.00ms which would be 13.16Hz!
	Average pre-processing time:	32.96ms which means	43.37 over the total.       
		Usually 20% higher than actual value, which would be 26.37ms
	Average voxel encoder time:	0.62ms which means	0.82 over the total.
	Average middle encoder time:	0.88ms which means	1.16 over the total.
	Average backbone time:		0.91ms which means	1.20 over the total.
	Average neck time:		0.27ms which means	0.35 over the total.
	Average post-processing time:	34.73ms which means	45.70 over the total.

By summing all the "sub-components" we get:      
	A total time of 70.38ms (14.21Hz) which means	92.60% over the total of	76.00ms      
and by considering the "real" value for the preprocessing (20% less), we get:      
	A total time of 63.79ms (15.68Hz) which means	83.93% over the total of	76.00ms      
while the inference (from voxel encoder to neck) accounts for:      
	A total time of 2.69ms which means		3.54% over the total of		76.00ms
